In [17]:
import pandas as pd
import numpy as np
import re

In [6]:
pays = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_all_years_consl.csv',nrows=10)

In [7]:
pays.head(2)

,payment_id,npi,fn,ln,address,city,state,zip,specialty,company_id,...,name_d2,name_d3,name_d4,name_d5,ndc_d1,ndc_d2,ndc_d3,ndc_d4,ndc_d5,year
0,235088.0,1174629083,FATIMA,ASGHAR,94 OLD SHORT HILLS RD,LIVINGSTON,NJ,7039,Allopathic & Osteopathic Physicians|Internal M...,100000005529,...,NaN,NaN,NaN,NaN,456120130,NaN,NaN,NaN,NaN,2013
1,334868.0,1386735850,RAGHUVEER,ANNAM,290 RED SCHOOL LN,PHILLIPSBURG,NJ,8865,Allopathic & Osteopathic Physicians|Internal M...,100000005529,...,NaN,NaN,NaN,NaN,456120130,NaN,NaN,NaN,NaN,2013


In [10]:
pays[:1].to_dict('index')

{0: {'address': '94 OLD SHORT HILLS RD',
  'amount': 14.83,
  'city': 'LIVINGSTON',
  'company': 'FOREST PHARMACEUTICALS',
  'company_id': 100000005529,
  'fn': 'FATIMA',
  'form': 'In-kind items and services',
  'ln': 'ASGHAR',
  'name_d1': 'LINZESS',
  'name_d2': nan,
  'name_d3': nan,
  'name_d4': nan,
  'name_d5': nan,
  'nature': 'Food and Beverage',
  'ndc_d1': 456120130,
  'ndc_d2': nan,
  'ndc_d3': nan,
  'ndc_d4': nan,
  'ndc_d5': nan,
  'npi': 1174629083,
  'payment_id': 235088.0,
  'phys_owns': 'No',
  'record_id': 100249796,
  'specialty': 'Allopathic & Osteopathic Physicians|Internal Medicine',
  'state': 'NJ',
  'year': 2013,
  'zip': 7039}}

In [11]:
meds = pd.read_csv('/Volumes/Seagate/Galvanize/Drug_Products_in_the_Medicaid_Drug_Rebate_Program.csv',
                  usecols=['NDC','Labeler Name','FDA Product Name','Year'])

In [12]:
meds.head(2)

,Year,Labeler Name,NDC,FDA Product Name
0,2014,ELI LILLY AND COMPANY,2120001,AMYVID ...
1,2014,ELI LILLY AND COMPANY,2140701,QUINIDINE GLUCONATE ...


In [23]:
#Removing excess spacing & any symbols
meds['FDA Product Name'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in meds['FDA Product Name']]
meds['FDA Product Name'] = meds['FDA Product Name'].str.strip()
meds['Labeler Name'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in meds['Labeler Name']]
meds['Labeler Name'] = meds['Labeler Name'].str.strip()

In [25]:
meds.head(2)

,Year,Labeler Name,NDC,FDA Product Name
0,2014,ELI LILLY AND COMPANY,2120001,AMYVID
1,2014,ELI LILLY AND COMPANY,2140701,QUINIDINE GLUCONATE


In [51]:
medsgrp = meds.groupby(['FDA Product Name','Labeler Name'])['NDC'].nunique().to_frame().reset_index()

In [52]:
print(len(medsgrp))
medsgrp.head()

45533


,FDA Product Name,Labeler Name,NDC
0,0000000000000000000000000000000000000ONIVYDE 4...,MERRIMACK PHARMACEUTICALS INC,1
1,0075 KCL IN 5 DEXTROSE AND 045 NACL INJECTION USP,B BRAUN MEDICAL INC,1
2,015 KCL IN 09 NACL INJECTION USP,B BRAUN MEDICAL INC,1
3,015 KCL IN 5 DEXTROSE AND 020 NACL INJECTION USP,B BRAUN MEDICAL INC,2
4,015 KCL IN 5 DEXTROSE AND 033 NACL INJECTION USP,B BRAUN MEDICAL INC,1


In [35]:
scripts = pd.read_csv('/volumes/Seagate/Galvanize/nj_scripts_all_years.csv',usecols=['drug_name','generic_name'])

In [36]:
#Only getting brand name drugs
scripts = scripts[scripts['drug_name']!=scripts['generic_name']]

In [37]:
scriptsgrp = scripts.groupby('drug_name')['generic_name'].nunique().to_frame().reset_index()

In [49]:
print(len(scriptsgrp))
scriptsgrp.head()

1448


,drug_name,generic_name
0,ABILIFY,1
1,ABILIFY DISCMELT,1
2,ABILIFY MAINTENA,1
3,ABRAXANE,1
4,ABSTRAL,1


In [53]:
common = scriptsgrp.merge(medsgrp,left_on='drug_name',right_on='FDA Product Name')

In [55]:
scripts[(~scripts.drug_name.isin(common.drug_name))]

,drug_name,generic_name
0,VIAGRA,SILDENAFIL
4,ANDROGEL,TESTOSTERONE
7,VESICARE,SOLIFENACIN SUCCINATE
12,FLOMAX,TAMSULOSIN
15,RAPAFLO,SILODOSIN
19,AVODART,DUTASTERIDE
20,TOVIAZ,FESOTERODINE FUMARATE
53,CELEBREX,CELECOXIB
55,LYRICA,PREGABALIN
58,LIDODERM,LIDOCAINE VISCOUS


In [56]:
medsgrp['VIAGRA']

,FDA Product Name,Labeler Name,NDC


In [76]:
medsgrp[medsgrp['FDA Product Name'].str.contains('VIAGRA')].groupby('Labeler Name')['NDC'].count()

,FDA Product Name,Labeler Name,NDC
43965,VIAGRA SILDENAFIL CITRATE,PFIZER INC,5
43966,VIAGRA SILDENAFIL CITRATE 100MG TAB,PFIZER INC,2
43967,VIAGRA SILDENAFIL CITRATE 50MG TAB,PFIZER INC,2


In [77]:
scriptsgrp.to_csv('/Volumes/Seagate/Galvanize/brand_drugs_scripts.csv',index=False)

In [78]:
medsgrp.to_csv('/Volumes/Seagate/Galvanize/medicare_drugs.csv',index=False)

# Start HERE

In [1]:
import pandas as pd
import numpy as np

In [2]:
scriptsgrp= pd.read_csv('/Volumes/Seagate/Galvanize/brand_drugs_scripts.csv')

In [3]:
medsgrp= pd.read_csv('/Volumes/Seagate/Galvanize/medicare_drugs.csv')

In [9]:
medsgrp[medsgrp['FDA Product Name'].str.contains('VIAGRA')].groupby('Labeler Name')['NDC'].count()

Labeler Name
PFIZER INC    3
Name: NDC, dtype: int64

Make a dict that has:

    keys = COMPANY
    values = ALL DRUGS
    
if the drug is in the k, v for that company:

    New Col [company] = Company

groupby company, iterate through company.unique, take company & index by it, for each company iterate through the drugs & add it to the values of that company

Map based on iff the drug is 

In [38]:
'VIAGRA'.str.contains('VIAGRA')

AttributeError: 'str' object has no attribute 'str'

In [5]:
viag = ['VIAGRA SILDENAFIL CITRATE','VIAGRA SILDENAFIL CITRATE 100MG TAB','VIAGRA SILDENAFIL CITRATE 50MG TAB']

In [7]:
viag

['VIAGRA SILDENAFIL CITRATE',
 'VIAGRA SILDENAFIL CITRATE 100MG TAB',
 'VIAGRA SILDENAFIL CITRATE 50MG TAB']

In [ ]:
lambda x: map(lambda w : v + w, y), x) 

for every drug:
    medsgrp[medsgrp['FDA Product Name'].str.contains(DRUG)].groupby('Labeler Name')['NDC'].count().to_frame().reset_index()

In [12]:
medsgrp[medsgrp['FDA Product Name'].str.contains('VIAGRA')].groupby('Labeler Name')['NDC'].count().to_frame().reset_index().sort_values('NDC',ascending=False)

,Labeler Name,NDC
0,PFIZER INC,3


In [35]:
x = 
        medsgrp[medsgrp['FDA Product Name'].str.contains('VIAGRA')].groupby('Labeler Name')['NDC'].count().to_frame(). \
        reset_index().sort_values('NDC',ascending=False).reset_index().drop('index',axis=1)['Labeler Name'][0]

In [42]:
d = {}
def company_drug(x):
    if x['drug_name'] not in d:
        d[x['drug_name']] = medsgrp[medsgrp['FDA Product Name'].str.contains(x['drug_name'])].groupby('Labeler Name')['NDC'].count().to_frame(). \
        reset_index().sort_values('NDC',ascending=False).reset_index().drop('index',axis=1)['Labeler Name'][0]

In [43]:
scriptsgrp.apply(company_drug,axis=1)

IndexError: ('index out of bounds', 'occurred at index 19')

In [45]:
medsgrp[medsgrp['FDA Product Name'].str.contains('ABSTRAL')].groupby('Labeler Name')['NDC'].count().to_frame(). \
        reset_index().sort_values('NDC',ascending=False).reset_index().drop('index',axis=1)['Labeler Name']

0        GALENA BIOPHARMA INC
1              PROSTRAKAN INC
2        SENTYNL THERAPEUTICS
3    SENTYNL THERAPEUTICS INC
Name: Labeler Name, dtype: object

In [47]:
medsgrp[medsgrp['FDA Product Name'].str.contains('ABSTRAL')]

,FDA Product Name,Labeler Name,NDC
231,ABSTRAL 100MCG,GALENA BIOPHARMA INC,1
232,ABSTRAL 100MCG,PROSTRAKAN INC,1
233,ABSTRAL 100MCG,SENTYNL THERAPEUTICS,1
234,ABSTRAL 100MCG,SENTYNL THERAPEUTICS INC,1
235,ABSTRAL 200MCG,GALENA BIOPHARMA INC,1
236,ABSTRAL 200MCG,PROSTRAKAN INC,1
237,ABSTRAL 200MCG,SENTYNL THERAPEUTICS,1
238,ABSTRAL 200MCG,SENTYNL THERAPEUTICS INC,1
239,ABSTRAL 300MCG,GALENA BIOPHARMA INC,1
240,ABSTRAL 300MCG,PROSTRAKAN INC,1
